In [1]:
import pandas as pd
import numpy as np
import yaml
from importlib import resources as impresources
from recurrent_health_events_prediction import configs

# Generalized Linear Mixed Effect Model

## Import Configs and Load Preprocessed Data

In [2]:
with open((impresources.files(configs) / 'data_config.yaml')) as f:
    data_config = yaml.safe_load(f)

In [3]:
dataset = "relapse"
training_data_config = data_config["training_data"][dataset]

In [4]:
from pathlib import Path

training_data_path = training_data_config["preprocessed_path"]
historcial_data_path = Path(training_data_path) / 'multiple_relapses_patients' / 'historical_relapses.csv'
last_data_path = Path(training_data_path) / 'multiple_relapses_patients' / 'last_relapses.csv'

In [5]:
last_events_df = pd.read_csv(last_data_path)
historical_events_df = pd.read_csv(historcial_data_path)

## Split Train-Test Data

In [6]:
if dataset == "mimic":
    features = [
        "HOSPITALIZATION_DAYS",
        "DAYS_IN_ICU",
        "MED_PAST_TIMES_TO_HOSP",
        "NUM_PREV_HOSPITALIZATIONS",
    ]
else:
    features = [
        "DONOR_ID",
        "NUM_PREV_RELAPSES",
        "PREV_NUM_DRUGS_POSITIVE",
        "DRUG_POSITIVE_PAST_SUM",
        "NUM_POSITIVES_SINCE_LAST_NEGATIVE",
        "LOG_PARTICIPATION_DAYS",
        "LOG_TIME_SINCE_LAST_NEGATIVE",
        "LOG_TIME_SINCE_LAST_POSITIVE",
        "AGE",
        "LOG_TIME_RELAPSE_PAST_MEDIAN",
        "LOG_TIME_RELAPSE_PAST_STD",
        "PREV_RELAPSE_30_DAYS",
        "RELAPSE_30_DAYS_PAST_SUM",
    ]
    # features_to_encode = ['PROGRAM_TYPE', 'RACE', 'GENDER']
    target_col = "RELAPSE_30_DAYS"
    # features_to_encode = ['PROGRAM_TYPE', 'RACE', 'GENDER']
    features_not_to_scale = ["PREV_RELAPSE_30_DAYS", "DONOR_ID"]
    # features += new_cols
    plot_shap = True
    missing_features = {
        "LOG_TIME_RELAPSE_PAST_STD": "median",
        "LOG_TIME_RELAPSE_PAST_MEDIAN": "median",
        "RELAPSE_30_DAYS_PAST_SUM": "median",
    }

In [7]:
X_train = historical_events_df[features]
y_train = historical_events_df[target_col]

X_test = last_events_df[features]
y_test = last_events_df[target_col]

In [8]:
from recurrent_health_events_prediction.training.utils_traditional_classifier import impute_missing_features, scale_features

X_train, X_test = impute_missing_features(X_train, X_test, missing_features)
X_train_scaled, X_test_scaled = scale_features(X_train, X_test, features_not_to_scale)

'NA' found in feature 'LOG_TIME_RELAPSE_PAST_STD'. Filling with strategy 'median'.
'NA' found in feature 'LOG_TIME_RELAPSE_PAST_MEDIAN'. Filling with strategy 'median'.
'NA' found in feature 'RELAPSE_30_DAYS_PAST_SUM'. Filling with strategy 'median'.
Scaling features: ['NUM_PREV_RELAPSES', 'PREV_NUM_DRUGS_POSITIVE', 'DRUG_POSITIVE_PAST_SUM', 'NUM_POSITIVES_SINCE_LAST_NEGATIVE', 'LOG_PARTICIPATION_DAYS', 'LOG_TIME_SINCE_LAST_NEGATIVE', 'LOG_TIME_SINCE_LAST_POSITIVE', 'AGE', 'LOG_TIME_RELAPSE_PAST_MEDIAN', 'LOG_TIME_RELAPSE_PAST_STD', 'RELAPSE_30_DAYS_PAST_SUM']


In [9]:
train_df = X_train_scaled.copy()
train_df[target_col] = y_train
test_df = X_test_scaled.copy()
test_df[target_col] = y_test

In [10]:
def split_data_by_subjects_with_duration_balance(training_df, duration_col, 
                                               new_subjects_ratio=0.05, known_subjects_ratio=0.3,
                                               subject_id_col = "SUBJECT_ID", event_id_col= "HADM_ID", random_state=None):
    """
    Splits data while maintaining similar average event duration across splits.
    
    Parameters:
        training_df (pd.DataFrame): Input dataframe
        duration_col (str): Column name containing duration/age information
        new_subjects_ratio (float): Ratio of new subjects for test
        known_subjects_ratio (float): Ratio of known subjects for event-level test
        subject_id_col (str): Column name for subject IDs
        event_id_col (str): Column name for event IDs
        random_state (int): Random seed
        
    Returns:
        tuple: (X_train, X_new_subjects_test, X_known_subjects_test)
    """
    
    # Input validation
    if not (0 <= new_subjects_ratio <= 1 and 0 <= known_subjects_ratio <= 1):
        raise ValueError("Ratios must be between 0 and 1")
    if new_subjects_ratio + known_subjects_ratio > 1:
        raise ValueError("Sum of ratios cannot exceed 1")
    
    np.random.seed(random_state)
    
    # Calculate subject-level duration statistics
    subject_stats = training_df.groupby(subject_id_col)[duration_col].agg(['mean', 'count'])
    subject_stats['duration_strata'] = pd.qcut(subject_stats['mean'], q=5, duplicates='drop')
    
    # Initialize containers
    new_test_subjects = []
    known_test_subjects = []
    train_subjects = []
    
    # Stratified splitting for new subjects
    for stratum in subject_stats['duration_strata'].unique():
        stratum_subjects = subject_stats[subject_stats['duration_strata'] == stratum].index.values
        np.random.shuffle(stratum_subjects)
        
        num_new = max(1, int(len(stratum_subjects) * new_subjects_ratio))
        new_test_subjects.extend(stratum_subjects[:num_new])
        train_subjects.extend(stratum_subjects[num_new:])
    
    # Create initial splits
    X_train = training_df[training_df[subject_id_col].isin(train_subjects)]
    X_new_subjects_test = training_df[training_df[subject_id_col].isin(new_test_subjects)].copy()
    
    # Stratified splitting for known subjects' events
    known_subject_stats = subject_stats.loc[train_subjects]
    for stratum in known_subject_stats['duration_strata'].unique():
        known_subjects_stats_more_than_one = known_subject_stats[known_subject_stats['count'] > 1]
        stratum_subjects = known_subjects_stats_more_than_one[known_subjects_stats_more_than_one['duration_strata'] == stratum].index.values
        np.random.shuffle(stratum_subjects)
        
        num_known = max(1, int(len(stratum_subjects) * known_subjects_ratio))
        selected = stratum_subjects[:num_known]
        known_test_subjects.extend(selected)
    
    # Select last event for known test subjects
    subject_events = X_train[X_train[subject_id_col].isin(known_test_subjects)]
    event_ids = subject_events.groupby(subject_id_col)[event_id_col].apply(
        lambda x: x.iloc[-1] if len(x) > 1 else None
    ).dropna()
    
    X_known_subjects_test = X_train[X_train[event_id_col].isin(event_ids)].copy()
    X_train = X_train[~X_train[event_id_col].isin(event_ids)]
    
    # Verify duration balance
    print("Duration balance verification:")
    for name, df in zip(['Train', 'New Subjects Test', 'Known Subjects Test'],
                        [X_train, X_new_subjects_test, X_known_subjects_test]):
        if len(df) > 0:
            print(f"{name}: Mean duration = {df[duration_col].mean():.2f}")
    
    return X_train, X_new_subjects_test, X_known_subjects_test

In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38183 entries, 0 to 38182
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   DONOR_ID                           38183 non-null  int64  
 1   NUM_PREV_RELAPSES                  38183 non-null  float64
 2   PREV_NUM_DRUGS_POSITIVE            38183 non-null  float64
 3   DRUG_POSITIVE_PAST_SUM             38183 non-null  float64
 4   NUM_POSITIVES_SINCE_LAST_NEGATIVE  38183 non-null  float64
 5   LOG_PARTICIPATION_DAYS             38183 non-null  float64
 6   LOG_TIME_SINCE_LAST_NEGATIVE       38183 non-null  float64
 7   LOG_TIME_SINCE_LAST_POSITIVE       38183 non-null  float64
 8   AGE                                38183 non-null  float64
 9   LOG_TIME_RELAPSE_PAST_MEDIAN       38183 non-null  float64
 10  LOG_TIME_RELAPSE_PAST_STD          38183 non-null  float64
 11  PREV_RELAPSE_30_DAYS               38183 non-null  int

In [12]:
train_df.to_csv("/workspaces/master-thesis-recurrent-health-events-prediction/data/data_for_R_code/relapse/train_df.csv", index=False)
test_df.to_csv("/workspaces/master-thesis-recurrent-health-events-prediction/data/data_for_R_code/relapse/test_df.csv", index=False)

## Log-Normal Mixed Effect Model

In [20]:
# --- Python code (run in your Python/Jupyter cell) ---
import pandas as pd
from rpy2 import robjects as ro
from rpy2.robjects import pandas2ri

# 1) Send pandas DataFrames to R -----------------------
with ro.conversion.localconverter(ro.default_converter + pandas2ri.converter):
    ro.globalenv["train_df"] = ro.conversion.py2rpy(train_df)
    ro.globalenv["test_df"]  = ro.conversion.py2rpy(test_df)

In [23]:
# --- Python: build explicit formula & run R via rpy2, then compute AUC in Python ---
from rpy2 import robjects as ro
from rpy2.robjects import pandas2ri
import numpy as np
from sklearn.metrics import roc_auc_score

# 0) Build an explicit formula to avoid "." expansion issues in R
response = "RELAPSE_30_DAYS"
groupvar = "DONOR_ID"
predictors = [c for c in train_df.columns if c not in (response, groupvar)]
fixed_rhs = " + ".join(predictors) if predictors else "1"
formula_str = f'{response} ~ {fixed_rhs} + (1 | {groupvar})'

# 1) Send data & formula into R
with ro.conversion.localconverter(ro.default_converter + pandas2ri.converter):
    ro.globalenv["train_df"] = ro.conversion.py2rpy(train_df)
    ro.globalenv["test_df"]  = ro.conversion.py2rpy(test_df)
ro.globalenv["formula_str"] = ro.StrVector([formula_str])

# 2) Fit, summarize, predict in R
ro.r(r"""
suppressPackageStartupMessages({
  if (!requireNamespace("lme4", quietly = TRUE)) install.packages("lme4")
  library(lme4)
})

# Coerce grouping var to factor and align levels
train_df$DONOR_ID <- factor(train_df$DONOR_ID)
test_df$DONOR_ID  <- factor(test_df$DONOR_ID, levels = levels(train_df$DONOR_ID))

# Ensure binary outcome is 0/1 integer
to_bin <- function(x) {
  if (is.logical(x)) return(as.integer(x))
  if (is.factor(x))  return(as.integer(x == levels(x)[2]))  # adjust if needed
  return(as.integer(x))
}
train_df$RELAPSE_30_DAYS <- to_bin(train_df$RELAPSE_30_DAYS)
test_df$RELAPSE_30_DAYS  <- to_bin(test_df$RELAPSE_30_DAYS)

# Build formula explicitly (no ".")
ff <- as.formula(formula_str)

# Fit the GLMM
m1 <- glmer(
  ff,
  data = train_df,
  family = binomial(link = "logit"),
  control = glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 2e5))
)

# Capture summary text
model_summary <- capture.output(summary(m1))

# Predict probabilities on test data
pred_probs <- predict(m1, newdata = test_df, type = "response", allow.new.levels = TRUE)
y_true <- test_df$RELAPSE_30_DAYS

# Drop any NA/Inf
keep <- is.finite(pred_probs) & !is.na(y_true)
pred_probs <- pred_probs[keep]
y_true <- y_true[keep]
""")

# 3) Bring results back to Python
with ro.conversion.localconverter(ro.default_converter + pandas2ri.converter):
    pred_probs = np.array(ro.globalenv["pred_probs"])
    y_true     = np.array(ro.globalenv["y_true"])
    summary_lines = list(ro.globalenv["model_summary"])

# 4) Print summary and compute AUC in Python
print("\n".join(summary_lines))
auc = roc_auc_score(y_true, pred_probs)
print("\nAUC:", auc)


R callback write-console: 
Correlation matrix not shown by default, as p = 13 > 12.
Use print(out$value, correlation=TRUE)  or
    vcov(out$value)        if you need it

  


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: RELAPSE_30_DAYS ~ NUM_PREV_RELAPSES + PREV_NUM_DRUGS_POSITIVE +  
    DRUG_POSITIVE_PAST_SUM + NUM_POSITIVES_SINCE_LAST_NEGATIVE +  
    LOG_PARTICIPATION_DAYS + LOG_TIME_SINCE_LAST_NEGATIVE + LOG_TIME_SINCE_LAST_POSITIVE +  
    AGE + LOG_TIME_RELAPSE_PAST_MEDIAN + LOG_TIME_RELAPSE_PAST_STD +  
    PREV_RELAPSE_30_DAYS + RELAPSE_30_DAYS_PAST_SUM + (1 | DONOR_ID)
   Data: train_df
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 2e+05))

      AIC       BIC    logLik -2*log(L)  df.resid 
  42171.1   42290.8  -21071.6   42143.1     38169 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-352.06   -0.85    0.41    0.65    2.18 

Random effects:
 Groups   Name        Variance Std.Dev.
 DONOR_ID (Intercept) 0.2089   0.4571  
Number of obs: 38183, groups:  DONOR_ID, 12289

Fixed effects:
                                   Estimate

In [ ]:
import statsmodels.formula.api as smf

# Fit linear mixed model with random intercept per patient
mixed_log_linear_model = smf.mixedlm("LOG_EVENT_DURATION ~ HOSPITALIZATION_DAYS + DAYS_IN_ICU + MED_PAST_TIMES_TO_HOSP + NUM_PREV_HOSPITALIZATIONS", 
                    X_train, 
                    groups=X_train["SUBJECT_ID"],)
result = mixed_log_linear_model.fit()
print(result.summary())

In [ ]:
mixed_log_linear_model.predict(X_test)